# Форматы данных (1)

Материалы:
* Макрушин С.В. "Лекция 4: Форматы данных"
* https://docs.python.org/3/library/json.html
* https://docs.python.org/3/library/pickle.html
* https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

1. Вывести все адреса электронной почты, содержащиеся в адресной книге `addres-book.json`

2. Вывести телефоны, содержащиеся в адресной книге `addres-book.json`

3. По данным из файла `addres-book-q.xml` сформировать список словарей с телефонами каждого из людей. 

## Лабораторная работа №4

### JSON

1.1 Считайте файл `dataibutors_sample.json`. Воспользовавшись модулем `json`, преобразуйте содержимое файла в соответствующие объекты python. Выведите на экран информацию о первых 3 пользователях.

In [1]:
import json
with open("contributors_sample.json", 'r') as file:
    data = json.load(file)
data[:3] #list

[{'username': 'uhebert',
  'name': 'Lindsey Nguyen',
  'sex': 'F',
  'address': '01261 Cameron Spring\nTaylorfurt, AK 97791',
  'mail': 'jsalazar@gmail.com',
  'jobs': ['Energy engineer',
   'Engineer, site',
   'Environmental health practitioner',
   'Biomedical scientist',
   'Jewellery designer'],
  'id': 35193},
 {'username': 'vickitaylor',
  'name': 'Cheryl Lewis',
  'sex': 'F',
  'address': '66992 Welch Brooks\nMarshallshire, ID 56004',
  'mail': 'bhudson@gmail.com',
  'jobs': ['Music therapist',
   'Volunteer coordinator',
   'Designer, interior/spatial'],
  'id': 91970},
 {'username': 'sheilaadams',
  'name': 'Julia Allen',
  'sex': 'F',
  'address': 'Unit 1632 Box 2971\nDPO AE 23297',
  'mail': 'darren44@yahoo.com',
  'jobs': ['Management consultant',
   'Engineer, structural',
   'Lecturer, higher education',
   'Theatre manager',
   'Designer, textile'],
  'id': 1848091}]

1.2 Выведите уникальные почтовые домены, содержащиеся в почтовых адресах людей

In [2]:
temp = set()
for items in data:
    t = items['mail'].split("@") #['andrew22', 'hotmail.com']
    temp.add(t[1])
print(temp)

{'gmail.com', 'yahoo.com', 'hotmail.com'}


1.3 Напишите функцию, которая по `username` ищет человека и выводит информацию о нем. Если пользователь с заданным `username` отсутствует, возбудите исключение `ValueError`

In [3]:
def f(data, name):
    try:
        for item in data:
            if item["name"] == name:
                return item
        else:
            raise ValueError
    except ValueError as e:
        print(f"Человек отсутствует{e}")

username = "Julia Allen"
f(data, username)

{'username': 'sheilaadams',
 'name': 'Julia Allen',
 'sex': 'F',
 'address': 'Unit 1632 Box 2971\nDPO AE 23297',
 'mail': 'darren44@yahoo.com',
 'jobs': ['Management consultant',
  'Engineer, structural',
  'Lecturer, higher education',
  'Theatre manager',
  'Designer, textile'],
 'id': 1848091}

1.4 Посчитайте, сколько мужчин и женщин присутсвует в этом наборе данных.

In [4]:
male_count = 0
female_count = 0
for item in data:
    if item["sex"] == 'F':
        female_count+=1
    else:
        male_count+=1
print(f"Мужчин - {male_count}, Женщин - {female_count}")

Мужчин - 2064, Женщин - 2136


1.5 Создайте `pd.DataFrame` `dataibutors`, имеющий столбцы `id`, `username` и `sex`.

In [5]:
import pandas as pd
pd_data = pd.DataFrame(data, columns=['id','username','sex'])
print(pd_data)

           id       username sex
0       35193        uhebert   F
1       91970    vickitaylor   F
2     1848091    sheilaadams   F
3       50969       nicole82   F
4      676820         jean67   M
...       ...            ...  ..
4195   423555  stevenspencer   F
4196    35251      rwilliams   M
4197   135887      lmartinez   F
4198   212714     brendahill   M
4199   344321       mistyray   F

[4200 rows x 3 columns]


1.6 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Объедините `recipes` с таблицей `dataibutors` с сохранением строк в том случае, если информация о человеке отсутствует в JSON-файле. Для скольких человек информация отсутствует? 

In [6]:
recipes = pd.read_csv('recipes_sample.csv')
new_data_json = pd.read_json("contributors_sample.json")

merged_data = pd.merge(left=recipes, right=new_data_json, on='id', how='left') #on - ключевая колонка

missing_data = merged_data[merged_data['username'].isnull()] #Дата с usernama==nan

print(f"Отсутствует информация для {len(missing_data)} человек")

Отсутствует информация для 29830 человек


### pickle

2.1 На основе файла `dataibutors_sample.json` создайте словарь следующего вида: 
```
{
    должность: [список username людей, занимавших эту должность]
}
```

In [17]:
dic = {}
for person in data:
    for i in person["jobs"]:
        if i not in dic:
            dic[i]=[]

for person in data:
    for job in person["jobs"]:
        dic[job].append(person["name"])
#dic

2.2 Сохраните результаты в файл `job_people.pickle` и в файл `job_people.json` с использованием форматов pickle и JSON соответственно. Сравните объемы получившихся файлов. При сохранении в JSON укажите аргумент `indent`.

In [8]:
import pickle
import os
with open("job_people.pickle", 'wb') as file:
    pickle.dump(dic, file)

with open("job_people.json", 'w') as file:
    json.dump(dic, file, indent=3)

pickle_size = os.path.getsize("job_people.pickle")
json_size = os.path.getsize("job_people.json")
print(f"Пикле меньше на {(json_size-pickle_size)/1024} KB")

Пикле меньше на 259.345703125 KB


2.3 Считайте файл `job_people.pickle` и продемонстрируйте, что данные считались корректно. 

In [18]:
with open("job_people.pickle", 'rb') as file:
    temp = pickle.load(file)
#temp

### XML

3.1 По данным файла `steps_sample.xml` сформируйте словарь с шагами по каждому рецепту вида `{id_рецепта: ["шаг1", "шаг2"]}`. Сохраните этот словарь в файл `steps_sample.json`

In [10]:
from bs4 import BeautifulSoup
import json

with open('steps_sample.xml', 'r') as xml_file:
    xml_data = xml_file.read()

soup = BeautifulSoup(xml_data, 'html.parser')

recipe_steps = {}


recipes = soup.find_all('recipe')

for recipe in recipes:
    recipe_id = recipe.find('id').get_text()
    steps = [step.get_text() for step in recipe.find_all('step')]  # Список шагов
    recipe_steps[recipe_id] = steps  # Добавляем список шагов в словарь по id рецепта

with open('steps_sample.json', 'w') as json_file:
    json.dump(recipe_steps, json_file, indent=3)

/usr/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


3.2 По данным файла `steps_sample.xml` сформируйте словарь следующего вида: `кол-во_шагов_в_рецепте: [список_id_рецептов]`

In [11]:
steps_count_dict = {}

for recipe in recipes:
    recipe_id = recipe.find('id').text
    steps_count = len(recipe.find_all('step'))  # Количество шагов в рецепте
    
    if steps_count in steps_count_dict: 
        steps_count_dict[steps_count].append(recipe_id) # Добавляем количество шагов и id
    else:
        steps_count_dict[steps_count] = [recipe_id]

with open('steps_count_dict.json', 'w') as json_file:
    json.dump(steps_count_dict, json_file, indent=4)

3.3 Получите список рецептов, в этапах выполнения которых есть информация о времени (часы или минуты). Для отбора подходящих рецептов обратите внимание на атрибуты соответствующих тэгов.

In [19]:
recipes_with_time_info = []

for recipe in recipes:
    recipe_id = recipe.find('id').text
    steps = recipe.find_all('step') 
    
    for step in steps: # Проверяем каждый этап на наличие времени
        if 'has_minutes' in step.attrs or 'has_hours' in step.attrs or 'has_seconds' in step.attrs: #аттрс - словарь с аттрибутами
            recipes_with_time_info.append(recipe_id)
            break  # Прерываем цикл, если информация о времени найдена

print("Список рецептов c информацией o времени в этапах выполнения:")
for recipe_id in recipes_with_time_info:
    #print(recipe_id)

SyntaxError: incomplete input (1425792382.py, line 14)

3.4 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Для строк, которые содержат пропуски в столбце `n_steps`, заполните этот столбец на основе файла  `steps_sample.xml`. Строки, в которых столбец `n_steps` заполнен, оставьте без изменений.

In [13]:
recipe_steps_count = {}

recipes = soup.find_all('recipe')

for recipe in recipes:
    recipe_id = int(recipe.find('id').text)
    steps_count = len(recipe.find_all('step'))
    recipe_steps_count[recipe_id] = steps_count

recipes_df = pd.read_csv('recipes_sample.csv')

recipes_df['n_steps'] = recipes_df['n_steps'].fillna(recipes_df['id'].replace(to_replace=recipe_steps_count)) #map(recipe_steps_count)

recipes_df

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,11.0,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,3.0,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,5.0,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,7.0,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,200862,2007-11-25,16.0,this is based on a french recipe but i changed...,10.0
29996,zwetschgenkuchen bavarian plum cake,386977,240,177443,2009-08-24,22.0,"this is a traditional fresh plum cake, thought...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,10.0,this is a traditional late summer early fall s...,NaN
29998,zydeco soup,486161,60,227978,2012-08-29,7.0,this is a delicious soup that i originally fou...,NaN


3.5 Проверьте, содержит ли столбец `n_steps` пропуски. Если нет, то преобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_filled_nsteps.csv`

In [14]:
flag = False
if recipes_df['n_steps'].isnull().any():
    print("Столбец 'n_steps' содержит пропуски.")
    flag = True
else:
    print("Столбец 'n_steps' не содержит пропуски.")

print(recipes_df['n_steps'].dtype) # уЖЕ ИНТ

if flag:
    recipes_df['n_steps'] = recipes_df['n_steps'].astype('int')

recipes_df.to_csv('recipes_sample_with_filled_nsteps.csv', index=False)

Столбец 'n_steps' не содержит пропуски.
float64
